# BlurNet



## Setup the environment

### Import all relevant modules

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!pip3 install dominate visdom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=1c685514333249ef698f1aef269d3d81a604807639c30b9a4b4c4a8f074775f7
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


### Import dataset from Google Drive & Format it

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_location = '/content/drive/MyDrive/CS188/cv-project-data.zip'
output_location = '/content/data'
!echo "unzipping..."
!rm -rf $output_location
!mkdir $output_location
!unzip -q $file_location -d $output_location
!echo "done!"

unzipping...
done!


### Clone the git repo & setup file structure



In [ ]:
!rm -rf blur-ai
!rm -rf tripathy
!git clone https://github.com/chaidhat/blur-ai.git

Cloning into 'blur-ai'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 96 (delta 16), reused 89 (delta 14), pack-reused 0
Receiving objects: 100% (96/96), 1.17 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (16/16), done.


### Execute the code

In [ ]:
!cd blur-ai/tripathy ; python3 train.py --dataroot ./datasets --model cycle_gan --dataset_mode unaligned --which_model_netG resnet_9blocks --which_direction AtoB --super_epoch 50 --super_epoch_start 0 --super_mode aligned --super_start 1 --name mygan_70 --no_dropout --niter 50

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
dataroot: ./datasets
dataset_mode: unaligned
display_freq: 100
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
epoch_count: 1
fineSize: 256
gpu_ids: [0]
identity: 0.5
init_type: normal
input_nc: 3
isTrain: True
lambda_A: 10.0
lambda_B: 10.0
loadSize: 286
lr: 0.0002
lr_decay_iters: 50
lr_policy: lambda
max_dataset_size: inf
model: cycle_gan
nThreads: 2
n_layers_D: 3
name: mygan_70
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_dropout: True
no_flip: False
no_html: False
no_lsgan: False
norm: instance
output_nc: 3
phase: train
pool_size: 50
print_freq: 100
resize_or_crop: resize_and_crop
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: False
super_epoch: 50
super_epoch_start: 0
super_mode: aligned
super_start: 1
update_html_freq: 1000
which_direction: AtoB
which_epoch: latest
which_model_netD: basic
which_model_netG: resnet_9blocks
--

In [ ]:
!cd blur-ai/tripathy ; python test.py --dataroot ./datasets --model cycle_gan --dataset_mode unaligned --which_model_netG resnet_9blocks --which_direction AtoB --name mygan_70 --how_many 100

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
dataroot: ./datasets
dataset_mode: unaligned
display_id: 1
display_port: 8097
display_winsize: 256
fineSize: 256
gpu_ids: [0]
how_many: 100
init_type: normal
input_nc: 3
isTrain: False
loadSize: 286
max_dataset_size: inf
model: cycle_gan
nThreads: 2
n_layers_D: 3
name: mygan_70
ndf: 64
ngf: 64
no_dropout: False
no_flip: False
norm: instance
ntest: inf
output_nc: 3
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
which_direction: AtoB
which_epoch: latest
which_model_netD: basic
which_model_netG: resnet_9blocks
-------------- End ----------------
CustomDatasetDataLoader
dataset [UnalignedDataset] was created
cycle_gan
initialization method [normal]
initialization method [normal]
Traceback (most recent call last):
  File "/content/blur-ai/tripathy/test.py", line 17, in <module>
    model = create_model(opt)
  File "/content/blur-ai/tripathy/models/model

# IGNORE BELOW

Seed code from the Assignments to reset seed

In [ ]:
# Also, seed everything for reproducibility
# code from https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964#file-seed_everything-py
def seed_everything(seed: int):
    import os
    import random
    import numpy as np
    import torch
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(0)

## Our model

In [ ]:
# Define your model.
class BlurNet(nn.Module):

    def __init__(self, mode='finetune', num_classes=100):
        super().__init__()
        """
        use the resnet18 model from torchvision models. Remember to set pretrained as true

        mode has two options:
        1) linear: For this model, we want to freeze resnet18 features, then train a linear
            classifier which takes the features before resnet.fc (we do not want
            the pretrained resnet.fc for 1000 classes). And then write our own FC layer: which takes in the features and
            output scores of size 100 (because we have 100 categories).
            Because we want to freeze resnet18 features, we have to iterate through parameters()
            of our model, and manually set some parameters to requires_grad = False
            Or use other methods to freeze the features
        2) finetune: Same as 1), except that we we do not need to freeze the features and
           can finetune on the pretrained resnet model.
        """
        ################# Your Implementations #################################
        self.resnet = models.resnext50_32x4d(pretrained=True)
        if mode == "linear":
          for name, param in self.resnet.named_parameters():
            if not name.startswith('fc'):
                param.requires_grad = False
        elif mode != "finetune":
          print("unrecognized mode!")

        self.resnet.fc = nn.Linear(2048, num_classes, device=device)
        ################# End of your Implementations ##########################

    def forward(self, x):
        ################# Your Implementations #################################
        x = self.resnet(x)
        ################# End of your Implementations ##########################
        return x

    def to(self, device):
        return self.resnet.to(device=device)


## Data Loading

In [ ]:
# Build train/val/test datasets and data loaders using the modules we built before.
import os

from torch.utils.data import DataLoader, Dataset


class MiniPlacesTest(Dataset):

    def __init__(self, root_dir, transform):
        """
        Initialize the MiniPlaces test dataset with the root directory for the images,
        an optional data transformation,


        Args:
            root_dir (str): Root directory for the MiniPlaces images.
            transform (callable, optional): Optional data transformation to apply to the images.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.filenames = []
        self.ids = []

        # You should read the test images from '${root_dir}/images/test' dir and save their filepaths and ids
        # For instance: the id of image 'test/00000001.jpg' should be '00000001' in string
        ################# Your Implementations #################################
        self.labels = []
        self.length = 0
        for filename in os.scandir(f'{root_dir}/images/test'):
          if filename.is_file():
            self.length += 1
            self.filenames.append(filename.name)
            self.ids.append(filename.name.split("/")[-1].split(".")[0])

        ################# End of your Implementations ##########################

    def __len__(self):
        """
        Return the number of images in the dataset.

        Returns:
            int: Number of images in the dataset.
        """
        dataset_len = 0
        ################# Your Implementations #################################
        # Return the number of images in the dataset
        dataset_len = self.length
        ################# End of your Implementations ##########################
        return dataset_len

    def __getitem__(self, idx):
        """
        Return a single image and its corresponding id when given an index.

        Args:
            idx (int): Index of the image to retrieve.

        Returns:
            tuple: Tuple containing the image and its id.
        """
        ################# Your Implementations #################################
        # Load and preprocess image using self.root_dir,
        # self.filenames[idx], and self.transform (if specified)

        pretransformed_image = Image.open(f'{root_dir}/data/images/test/{self.filenames[idx]}')
        image = data_transform(pretransformed_image)
        id = self.ids[idx]
        ################# End of your Implementations ##########################
        return image, id


data_root = os.path.join(root_dir, 'data')
# Create MiniPlaces test dataset object
miniplaces_test = MiniPlacesTest(data_root, transform=data_transform)

# Create DataLoader for test set
test_loader = DataLoader(miniplaces_test,
                         batch_size=batch_size,
                         num_workers=num_workers,
                         shuffle=False)

NameError: name 'root_dir' is not defined


#Citation

@article{tripathy+kannala+rahtu,
  title={Learning image-to-image translation using paired and unpaired training samples},
  author={Tripathy, Soumya and Kannala, Juho and Rahtu, Esa},
  journal={arXiv preprint arXiv:1805.03189},
  year={2018}
}